In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [3]:
# Create a dash application
app = dash.Dash(__name__)


In [4]:
# Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
app.layout = html.Div(children=[
       html.H1('SpaceX Launch Records Dashboard',
              style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
       
       dcc.Dropdown(
              id='site-dropdown',
              options=[
              {'label': 'All Sites', 'value': 'ALL'},
              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
              ],
              value='ALL',
              placeholder='Select a Launch Site',
              searchable=False
       ),
       
       html.Br(),
       
       html.Div(dcc.Graph(id='success-pie-chart')),
       
       html.Br(),
       html.P("Payload range (Kg):"),
       
       dcc.RangeSlider(
              id='payload-slider',
              min=min_payload,
              max=max_payload,
              step=1000,
              value=[min_payload, max_payload],
              marks={
              0: {'label': str(min_payload)},
              2500: {'label': '2500'},
              5000: {'label': '5000'},
              7500: {'label': '7500'},
              10000: {'label': str(max_payload)}
              }
       ),
       
       html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])


In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(site):
    if site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Success Rate for All Launch Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site]
        fig = px.pie(filtered_df, values='class', names='Unnamed: 0', title=f'Success Rate for {site}')
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(site, payload_range):
    lower_payload, upper_payload = payload_range
    if site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= lower_payload) & (spacex_df['Payload Mass (kg)'] <= upper_payload)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                        title='Payload Mass vs. Launch Success for All Launch Sites', 
                        labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Launch Success'})
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == site) & (spacex_df['Payload Mass (kg)'] >= lower_payload) 
                                & (spacex_df['Payload Mass (kg)'] <= upper_payload)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                        title=f'Payload Mass vs. Launch Success for {site}', 
                        labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Launch Success'})
    return fig

In [6]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

With the completed dashboard, you can now easily analyze the SpaceX launch data and answer the following questions:

Which site has the largest successful launches?

You can refer to the pie chart titled "Success Rate for All Launch Sites" to see the number of successful launches for each site.
Which site has the highest launch success rate?

You can refer to the pie chart titled "Success Rate for All Launch Sites" to see the launch success rate for each site.
Which payload range(s) has the highest launch success rate?

You can use the scatter plot titled "Payload Mass vs. Launch Success for All Launch Sites" and analyze the data points to determine the payload range(s) with the highest launch success rate. You can hover over the data points to see the specific payload mass and launch success.
Which payload range(s) has the lowest launch success rate?

Similarly, you can analyze the scatter plot titled "Payload Mass vs. Launch Success for All Launch Sites" and identify the payload range(s) with the lowest launch success rate by examining the data points.
Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

You can refer to the scatter plot titled "Payload Mass vs. Launch Success for All Launch Sites" and use the color coding to identify the F9 Booster version with the highest launch success rate. The legend in the plot will provide the information about the Booster version corresponding to each color.
By interacting with the dropdown menus and slider on the dashboard, you can filter the data and obtain more specific insights based on your requirements.